In [4]:
pip install torchvision 

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 1.9 MB/s eta 0:00:01
   ----- ---------------------------------- 0.2/1.2 MB 2.3 MB/s eta 0:00:01
   ------------ --------------------------- 0.4/1.2 MB 3.4 MB/s eta 0:00:01
   ------------------------ --------------- 0.7/1.2 MB 4.6 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 5.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/159.8 MB ? eta -:--:--
   ---------------------------------------- 0.5/159.8 MB 11.3 MB/s eta 0:00:15
   ---------------------------------------- 0.9/159.8 MB 11.2 MB/s eta 0:00:15
   ---------------------------------------- 0.9/159.8 MB 11.2 MB/s eta 0:00:15
   ---------------------------------------- 1.9/159.8 MB 11.2 MB/s eta 0:00:15
    --------------------------------------- 2.3/159.8 MB 10.2 MB/s eta 0:00:16
    --------------------------------------- 2.8/159.8 MB 10.5 MB/s eta 0:00:15
    --

In [8]:
pip install opencv-python

   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.6 MB 1.3 MB/s eta 0:00:31
   ---------------------------------------- 0.1/38.6 MB 1.8 MB/s eta 0:00:22
   ---------------------------------------- 0.4/38.6 MB 3.3 MB/s eta 0:00:12
    --------------------------------------- 0.8/38.6 MB 4.6 MB/s eta 0:00:09
   - -------------------------------------- 1.3/38.6 MB 6.5 MB/s eta 0:00:06
   - -------------------------------------- 1.9/38.6 MB 7.0 MB/s eta 0:00:06
   -- ------------------------------------- 2.3/38.6 MB 8.0 MB/s eta 0:00:05
   -- ------------------------------------- 2.3/38.6 MB 8.0 MB/s eta 0:00:05
   --- ------------------------------------ 3.2/38.6 MB 8.1 MB/s eta 0:00:05
   --- ------------------------------------ 3.7/38.6 MB 8.2 MB/s eta 0:00:05
   --- ------------------------------------ 3.8/38.6 MB 8.3 MB/s eta 0:00:05
   ---- ----------------------------------- 4.8/38.6 MB 8.8 MB/s eta 0:00:04
   ---

In [9]:
import torch
from PIL import Image, ImageDraw, ImageFont
from torchvision import transforms, models
import torch.nn.functional as F

def prediction(image_path, yolo_model_path, classification_model_path, confidence_threshold=0.5, 
               colors=[(0, 255, 0), (255, 0, 0)], thickness=3, scale=1.0,
               show_names=True, show_confidence=True, num_classes=2, 
               text_size=15, text_color=(255, 255, 255), show_probabilities=True):
    def load_classification_model(model_path, num_classes):
        model = models.resnet18(pretrained=False)
        num_ftrs = model.fc.in_features
        model.fc = torch.nn.Linear(num_ftrs, num_classes)
        model.load_state_dict(torch.load(model_path))
        model.eval()
        return model

    def classify_image(image, model, transform, device):
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(image)
            probabilities = F.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
        return preds.item(), probabilities.squeeze().cpu().numpy()

    # загрузка фото
    image = Image.open(image_path)

    # загрузка модели YOLOv5
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_model_path)

    # Установка порога уверенности (от 0 до 1)
    model.conf = confidence_threshold

    # Предсказание на изображении
    results = model(image)

    # Загрузка классификационной модели
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification_model = load_classification_model(classification_model_path, num_classes).to(device)
    
    # Подготовка трансформации для классификационной модели
    classification_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Классификация изображения
    class_idx, probabilities = classify_image(image, classification_model, classification_transform, device)
    class_names = ['1-464 A-1', '1-464 A-2']  # Замените на свои классы
    classification_text = f"{class_names[class_idx]}: {probabilities[class_idx]:.2f}"

    # Загрузка шрифта и установка размера текста для детекции
    detection_font_size = int(15 * scale)
    try:
        detection_font = ImageFont.truetype("arial.ttf", detection_font_size)
    except IOError:
        detection_font = ImageFont.load_default()

    # Установка размера текста для классификации
    class_font_size = int(text_size)
    try:
        class_font = ImageFont.truetype("arial.ttf", class_font_size)
    except IOError:
        class_font = ImageFont.load_default()

    # Отображение результатов YOLOv5
    if len(results.xyxy[0]) > 0:
        draw = ImageDraw.Draw(image)
        for detection in results.xyxy[0]:
            label = int(detection[-1])
            confidence = detection[-2]
            box = detection[:-2]
            box = [(int(coord)) for coord in box]
            box = tuple(box)
            if label < len(colors):
                color = colors[label]
            else:
                color = (0, 255, 0)  # default color
            draw.rectangle(box, outline=color, width=thickness)
            if show_names:
                label_name = model.model.names[label]
                text = f"{label_name}"
                if show_confidence:
                    text += f" {confidence:.2f}"
                draw.text((box[0], box[1]), text, fill=color, font=detection_font)
        
        # Вывод классификационного текста в углу изображения
        if show_probabilities:
            prob_texts = [f"{class_names[i]}: {prob:.2f}" for i, prob in enumerate(probabilities)]
            for i, prob_text in enumerate(prob_texts):
                draw.text((10, 10 + i * (class_font_size + 5)), prob_text, fill=text_color, font=class_font)
        else:
            draw.text((10, 10), classification_text, fill=text_color, font=class_font)
        
        image.show()
    else:
        print("На фото нет объектов")

In [9]:
import torch
from PIL import Image, ImageDraw, ImageFont
from torchvision import transforms, models
import torch.nn.functional as F

def prediction(image_path, yolo_model_path, classification_model_path, confidence_threshold=0.5, 
               colors=[(0, 255, 0), (255, 0, 0)], thickness=3, scale=1.0,
               show_names=True, show_confidence=True, num_classes=2, 
               text_size=15, text_color=(255, 255, 255), show_probabilities=True):
    def load_classification_model(model_path, num_classes):
        model = models.resnet18(pretrained=False)
        num_ftrs = model.fc.in_features
        model.fc = torch.nn.Linear(num_ftrs, num_classes)
        model.load_state_dict(torch.load(model_path))
        model.eval()
        return model

    def classify_image(image, model, transform, device):
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(image)
            probabilities = F.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
        return preds.item(), probabilities.squeeze().cpu().numpy()

    # загрузка фото
    image = Image.open(image_path)

    # загрузка модели YOLOv5
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_model_path)

    # Установка порога уверенности (от 0 до 1)
    model.conf = confidence_threshold

    # Предсказание на изображении
    results = model(image)

    # Загрузка классификационной модели
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification_model = load_classification_model(classification_model_path, num_classes).to(device)
    
    # Подготовка трансформации для классификационной модели
    classification_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Классификация изображения
    class_idx, probabilities = classify_image(image, classification_model, classification_transform, device)
    class_names = ['1-464 A-1', '1-464 A-2']  # Замените на свои классы
    classification_text = f"{class_names[class_idx]}: {probabilities[class_idx]:.2f}"

    # Загрузка шрифта и установка размера текста для детекции
    detection_font_size = int(15 * scale)
    try:
        detection_font = ImageFont.truetype("arial.ttf", detection_font_size)
    except IOError:
        detection_font = ImageFont.load_default()

    # Установка размера текста для классификации
    class_font_size = int(text_size)
    try:
        class_font = ImageFont.truetype("arial.ttf", class_font_size)
    except IOError:
        class_font = ImageFont.load_default()

    # Отображение результатов YOLOv5
    if len(results.xyxy[0]) > 0:
        draw = ImageDraw.Draw(image)
        for detection in results.xyxy[0]:
            label = int(detection[-1])
            confidence = detection[-2]
            box = detection[:-2]
            box = [(int(coord)) for coord in box]
            box = tuple(box)
            if label < len(colors):
                color = colors[label]
            else:
                color = (0, 255, 0)  # default color
            draw.rectangle(box, outline=color, width=thickness)
            if show_names:
                label_name = model.model.names[label]
                text = f"{label_name}"
                if show_confidence:
                    text += f" {confidence:.2f}"
                draw.text((box[0], box[1]), text, fill=color, font=detection_font)
        
        # Вывод классификационного текста в углу изображения
        if show_probabilities:
            prob_texts = [f"{class_names[i]}: {prob:.2f}" for i, prob in enumerate(probabilities)]
            for i, prob_text in enumerate(prob_texts):
                draw.text((10, 10 + i * (class_font_size + 5)), prob_text, fill=text_color, font=class_font)
        else:
            draw.text((10, 10), classification_text, fill=text_color, font=class_font)
        
        image.show()
    else:
        print("На фото нет объектов")

In [9]:
import torch
from PIL import Image, ImageDraw, ImageFont
from torchvision import transforms, models
import torch.nn.functional as F

def prediction(image_path, yolo_model_path, classification_model_path, confidence_threshold=0.5, 
               colors=[(0, 255, 0), (255, 0, 0)], thickness=3, scale=1.0,
               show_names=True, show_confidence=True, num_classes=2, 
               text_size=15, text_color=(255, 255, 255), show_probabilities=True):
    def load_classification_model(model_path, num_classes):
        model = models.resnet18(pretrained=False)
        num_ftrs = model.fc.in_features
        model.fc = torch.nn.Linear(num_ftrs, num_classes)
        model.load_state_dict(torch.load(model_path))
        model.eval()
        return model

    def classify_image(image, model, transform, device):
        image = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            outputs = model(image)
            probabilities = F.softmax(outputs, dim=1)
            _, preds = torch.max(outputs, 1)
        return preds.item(), probabilities.squeeze().cpu().numpy()

    # загрузка фото
    image = Image.open(image_path)

    # загрузка модели YOLOv5
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=yolo_model_path)

    # Установка порога уверенности (от 0 до 1)
    model.conf = confidence_threshold

    # Предсказание на изображении
    results = model(image)

    # Загрузка классификационной модели
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    classification_model = load_classification_model(classification_model_path, num_classes).to(device)
    
    # Подготовка трансформации для классификационной модели
    classification_transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Классификация изображения
    class_idx, probabilities = classify_image(image, classification_model, classification_transform, device)
    class_names = ['1-464 A-1', '1-464 A-2']  # Замените на свои классы
    classification_text = f"{class_names[class_idx]}: {probabilities[class_idx]:.2f}"

    # Загрузка шрифта и установка размера текста для детекции
    detection_font_size = int(15 * scale)
    try:
        detection_font = ImageFont.truetype("arial.ttf", detection_font_size)
    except IOError:
        detection_font = ImageFont.load_default()

    # Установка размера текста для классификации
    class_font_size = int(text_size)
    try:
        class_font = ImageFont.truetype("arial.ttf", class_font_size)
    except IOError:
        class_font = ImageFont.load_default()

    # Отображение результатов YOLOv5
    if len(results.xyxy[0]) > 0:
        draw = ImageDraw.Draw(image)
        for detection in results.xyxy[0]:
            label = int(detection[-1])
            confidence = detection[-2]
            box = detection[:-2]
            box = [(int(coord)) for coord in box]
            box = tuple(box)
            if label < len(colors):
                color = colors[label]
            else:
                color = (0, 255, 0)  # default color
            draw.rectangle(box, outline=color, width=thickness)
            if show_names:
                label_name = model.model.names[label]
                text = f"{label_name}"
                if show_confidence:
                    text += f" {confidence:.2f}"
                draw.text((box[0], box[1]), text, fill=color, font=detection_font)
        
        # Вывод классификационного текста в углу изображения
        if show_probabilities:
            prob_texts = [f"{class_names[i]}: {prob:.2f}" for i, prob in enumerate(probabilities)]
            for i, prob_text in enumerate(prob_texts):
                draw.text((10, 10 + i * (class_font_size + 5)), prob_text, fill=text_color, font=class_font)
        else:
            draw.text((10, 10), classification_text, fill=text_color, font=class_font)
        
        image.show()
    else:
        print("На фото нет объектов")

## парамтры функции
1. **`image_path`**: Путь к изображению, на котором будут выполняться детекция объектов и классификация.

2. **`yolo_model_path`**: Путь к сохраненной модели YOLOv5 для детекции объектов.

3. **`classification_model_path`**: Путь к сохраненной модели классификации (ResNet или другой), которая будет использоваться для классификации всего изображения.

4. **`confidence_threshold`**: Порог уверенности (от 0 до 1) для детекции объектов. Объекты с уверенностью ниже этого порога будут игнорироваться.

5. **`colors`**: Список цветов (в формате RGB), используемых для обводки обнаруженных объектов. Каждый цвет соответствует определенному классу объектов.

6. **`thickness`**: Толщина линии, используемой для обводки обнаруженных объектов.

7. **`scale`**: Масштаб для размера текста, используемого для отображения названий и уверенностей обнаруженных объектов.

8. **`show_names`**: Булевый параметр, указывающий, следует ли отображать названия классов для обнаруженных объектов.

9. **`show_confidence`**: Булевый параметр, указывающий, следует ли отображать уверенность (в процентах) для обнаруженных объектов.

10. **`num_classes`**: Количество классов в модели классификации.

11. **`text_size`**: Размер текста для отображения результатов классификации (название класса и уверенность).

12. **`text_color`**: Цвет текста (в формате RGB) для отображения результатов классификации.

13. **`show_probabilities`**: Булевый параметр, указывающий, следует ли отображать вероятности для всех классов классификации. Если `True`, отображается список классов с вероятностями; если `False`, отображается только спрогнозированный класс с его вероятностью.

In [1]:
# Пример использования
image_path = "test1.jpg"
yolo_model_path = r"detect.pt"
classification_model_path = "class.pth"

# Задаем цвета для каждого класса (здесь для двух классов)
colors = [(0, 255, 0), (255, 0, 0)]
prediction(image_path, yolo_model_path, classification_model_path, confidence_threshold=0.5, 
           colors=colors, thickness=3, scale=1.0,
           show_names=True, show_confidence=True,
           num_classes=2, text_size=50, text_color=(255, 255, 0), show_probabilities=True)

# открывает приложение для просмотра фото по умлочанию

NameError: name 'prediction' is not defined